In [1]:
import area_mandelbrot as am
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [18]:
rng = np.random.default_rng()
bound = -2, 0.5, -2, 2
area_total = (np.abs(bound[0]) + bound[1]) * (np.abs(bound[2]) + bound[3])
s = 10 ** 6
i = 4 * 10 ** 3

samples = am.random_samples(rng, bound, s)
res = am.mandelbrot(samples, i)
ct_res = Counter(res)

In [19]:
area_mandel = area_total * (ct_res[i-1] / (sum(ct_res.values()) - ct_res[i-1]))
area_mandel

1.7773422778322154